### Load packages and sample data

In [1]:
import shlex
import re
import pandas as pd
import numpy as np
from typing import List
import utils

In [2]:
fields_to_include = ["OWNER","TITLE", "ABSTRACT", "SERVICETYPE", "SERVICELINK", "KEYWORDS"]
fields_to_output = ["OWNER","TITLE", "SERVICETYPE", "SERVICELINK"]
url_github_repo = "https://github.com/davidoesch/geoservice_harvester_poc/blob/main/data/"
url_github_repo_suffix = "?raw=true"
url_geoservices_CH_csv = "{}geoservices_CH.csv{}".format(url_github_repo,url_github_repo_suffix)
a = 'Strassen auswertung system'

In [3]:
def split_delimiters(word_list_with_delimiters: List[str]) -> List[str]:
    """Take care of left over delimiters, split strings even if in qoutes
        Return a list of words """
    delimiters = [";", ","]

    new_word_list = []

    for word in word_list_with_delimiters:
        if (any(delimiter in word for delimiter in delimiters)):
            splitted_words = re.split(r',|;', word)
            for splitted_word_ in splitted_words:
                new_word_list.append(splitted_word_)
        else:
            new_word_list.append(word)
    return new_word_list

def load_data():
    dataframe = pd.read_csv(url_geoservices_CH_csv, usecols=fields_to_include)
    return dataframe

In [4]:
res = split_delimiters(shlex.split(a))
dataframe_some_cols = load_data()
result = dataframe_some_cols[dataframe_some_cols.apply(lambda dataset: dataset.astype(str).str.contains(res[0], case=False).any(), axis=1)]
result =result.fillna("empty")

### TF-IDF ranking of the abstract

In [9]:
result['tfidf'] = result.apply(lambda row: utils.ranking_tfidf(row['ABSTRACT']), axis=1)


In [19]:
result.head()

,OWNER,TITLE,ABSTRACT,KEYWORDS,SERVICELINK,SERVICETYPE,tfidf
0,KT_AI,Abwasser Werkplan Gde,Diese Karte zeigt die Werkleitungen der Abwass...,empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(sowi, 0.54), (abwasserentsorgung, 0.5), (ka..."
65,KT_AI,Hanglagen CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(hangneigungsklassen, 0.88), (datensatz, 0.8..."
66,KT_AI,Hauptstrassennetz CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(strassen, 0.78), (wm, 0.7), (hauptstrassen,..."
68,KT_AI,Höhenkurven Kt,Diese Karte beinhaltet die Höhenkurven mit ein...,empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(höhenkurven, 0.84), (auswertung, 0.47), (be..."
70,KT_AI,IVS Geländekarte CH,"Geodaten des Bundesamtes für Strassen, als WMS...",empty,https://www.geoportal.ch/services/wms/ktai?ser...,WMS,"([(viv, 0.85), (geführt, 0.77), (wm, 0.71), (k..."


In [16]:
id = 70
result.loc[id]['ABSTRACT'], result.loc[id]['tfidf']

('Geodaten des Bundesamtes für Strassen, als WMS der swisstopo bezogen. Die Geländekarte stellt die kartografische Grundlage der Ersterfassung des Inventars dar. Sie beinhaltet ergänzende Angaben zu den Objekten nach Artikel 3 der Verordnung über das Bundesinventar der historischen Verkehrswege der Schweiz VIVS sowie zu den Objekten von regionaler und lokaler Bedeutung. Die Geländekarte wird vorerst nicht nachgeführt und deshalb nur als Zusatzinformation in der elektronischen Publikation sowie als gesonderte Karte in der gedruckten Ausgabe (vgl. Art. 4 VIVS) geführt. Der Kanton Solothurn hat für die Neubeurteilung der Objekte von regionaler/lokaler Bedeutung die Geländekarten überarbeitet. Diese überarbeiteten Versionen sind im vorliegenden Datensatz enthalten. WMS Service Geoportal - Kanton Appenzell Innerrhoden',
 ([('viv', 0.85),
   ('geführt', 0.77),
   ('wm', 0.71),
   ('kanton', 0.65),
   ('geländekart', 0.55),
   ('bedeutung', 0.5),
   ('lokal', 0.5),
   ('objekten', 0.49),
   (

In [17]:
result.loc[id]['ABSTRACT'], result.loc[id]['tfidf'], result.loc[id]['KEYWORDS'], result.loc[id]['TITLE']

('Geodaten des Bundesamtes für Strassen, als WMS der swisstopo bezogen. Die Geländekarte stellt die kartografische Grundlage der Ersterfassung des Inventars dar. Sie beinhaltet ergänzende Angaben zu den Objekten nach Artikel 3 der Verordnung über das Bundesinventar der historischen Verkehrswege der Schweiz VIVS sowie zu den Objekten von regionaler und lokaler Bedeutung. Die Geländekarte wird vorerst nicht nachgeführt und deshalb nur als Zusatzinformation in der elektronischen Publikation sowie als gesonderte Karte in der gedruckten Ausgabe (vgl. Art. 4 VIVS) geführt. Der Kanton Solothurn hat für die Neubeurteilung der Objekte von regionaler/lokaler Bedeutung die Geländekarten überarbeitet. Diese überarbeiteten Versionen sind im vorliegenden Datensatz enthalten. WMS Service Geoportal - Kanton Appenzell Innerrhoden',
 ([('viv', 0.85),
   ('geführt', 0.77),
   ('wm', 0.71),
   ('kanton', 0.65),
   ('geländekart', 0.55),
   ('bedeutung', 0.5),
   ('lokal', 0.5),
   ('objekten', 0.49),
   (

### Improvement of the TF-IDF with BM25 to execute a query on the text

In [5]:
# abstracts = result['ABSTRACT'].values.tolist()

bm25 = utils.TFIDF_BM25()
results = bm25.cleansing_ranking(result) # 1421 lines in 36 s

In [6]:
# results[15]['word'].tolist()
# [' '.join(abstract['word'].tolist()) for abstract in results]

bm25.fit(results)
res = bm25.transform('Brandmeldeanlage mit Produkte')

  (0, 2585)	1.0


In [7]:
np.where(res > 0)[0].tolist()
#filter(lambda x: x)

[543, 544, 545]

In [25]:
result.index.values[1]

65

In [17]:
results[543]

,word,tfidf
0,gewasserraum,2.23
1,wurd,1.92
2,geoprodukt,1.49
3,grundbuchkreis,1.47
4,nutzungseinschrank,1.38
...,...,...
107,20,0.16
108,gewasserschutzverordn,0.16
109,gschv,0.16
110,national,0.16
